<a href="https://colab.research.google.com/github/Riskaseptianaa/Images-Retrieval-140-175/blob/main/CBIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import scipy
from IPython import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import KFold
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
import xgboost as xgb
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/TKC-new"

In [4]:
%cd /content/gdrive/My Drive/TKC-new

/content/gdrive/My Drive/TKC-new


In [5]:
!ls

flowers  flowers.zip


In [5]:
DATADIR="/content/gdrive/MyDrive/TKC-new/flowers"
CATEGORIES=["flowers"]
training_data=[]
i = 1
for category in CATEGORIES:
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array=cv2.imread(os.path.join(path,img))
            new_image=np.asarray(cv2.resize(img_array,(300,300)))
            training_data.append([new_image, class_num])
            print("Gambar termuat : ",i)
            display.clear_output(wait=True)
            i=i+1
        except Exception as e:
            pass## Load Image Data

Gambar termuat :  100


## Splitting

In [6]:
X = []
y = []
for image,label in training_data:
    X.append(image)
    y.append(label)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)

In [8]:
print("Jumlah data latih : ",np.shape(X_train)[0])
print("Jumlah data uji   : ",np.shape(X_test)[0]) 

Jumlah data latih :  80
Jumlah data uji   :  20


## Preprocessing

In [9]:
def preprocessing1(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        img=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        arr_prep.append(img)
    return arr_prep

def preprocessing2(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        image=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        arr_prep.append(image)
    return arr_prep

def preprocessing3(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        image=cv2.cvtColor(arr[i], cv2.COLOR_BGR2HSV)
        arr_prep.append(image)
    return arr_prep

## Feature Extraction

In [10]:
def FtrExtractHOG(img):
    ftr,_=hog(img, orientations=9, pixels_per_cell=(50, 50),
            cells_per_block=(2, 2), visualize=True, multichannel=False, block_norm= 'L2')
    return ftr

def FtrExtractGLCM(img, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
        
    return feature

def FtrExtractHSV(img):
    chans = cv2.split(img)
    features = []

    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        features.extend(hist)
    return np.array(features).flatten()

In [11]:
def featureExtraction1(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractHOG(arr[i]))
    return arr_feature

properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
def featureExtraction2(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractGLCM(arr[i],props=properties))
    return arr_feature

def featureExtraction3(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractHSV(arr[i]))
    return arr_feature

## K-Fold Cross Validation

In [12]:
def kfold_1(X_train,y_train,K,stat):

  cvscores = []
  i = 1
  skf = KFold(n_splits=K)

  X_train_ = np.array(X_train)
  y_train = np.array(y_train)
  if (stat==1):
      RF = RandomForestClassifier(n_estimators=400, random_state = 42)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        RF.fit(x_train_new, y_train_new)

        y_pred = RF.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)
  elif (stat==2):
      DTL = DecisionTreeClassifier(criterion="entropy", min_samples_split=20, random_state=42)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        DTL.fit(x_train_new, y_train_new)

        y_pred = DTL.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)

  elif (stat==3):
      XG = xgb.XGBClassifier(subsample=0.8, learning_rate=0.1)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        XG.fit(x_train_new, y_train_new)

        y_pred = XG.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)
  
  return cvscores

In [13]:
X_trainp_shape=preprocessing1(X_train)
X_testp_shape=preprocessing1(X_test)

X_trainp_texture=preprocessing2(X_train)
X_testp_texture=preprocessing2(X_test)

X_trainp_color=preprocessing3(X_train)
X_testp_color=preprocessing3(X_test)

In [14]:
X_trainftr_shape=featureExtraction1(X_trainp_shape)
X_testftr_shape=featureExtraction1(X_testp_shape)

X_trainftr_texture=featureExtraction2(X_trainp_texture)
X_testftr_texture=featureExtraction2(X_testp_texture)

X_trainftr_color=featureExtraction3(X_trainp_color)
X_testftr_color=featureExtraction3(X_testp_color)

In [15]:
X_trainftr_combined = np.concatenate((X_trainftr_shape,X_trainftr_texture,X_trainftr_color),axis=1)
X_testftr_combined = np.concatenate((X_testftr_shape,X_testftr_texture,X_testftr_color),axis=1)

In [16]:
print("Dimensi fitur bentuk  : ",np.shape(X_trainftr_shape)[1])
print("Dimensi fitur tekstur : ",np.shape(X_trainftr_texture)[1])
print("Dimensi fitur warna   : ",np.shape(X_trainftr_color)[1])
print("Dimensi fitur gabungan: ",np.shape(X_trainftr_combined)[1])

Dimensi fitur bentuk  :  900
Dimensi fitur tekstur :  24
Dimensi fitur warna   :  768
Dimensi fitur gabungan:  1692


## CNN

In [26]:
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')

mean_image = np.mean(X_train, axis = 0)

X_train -= mean_image
X_test -= mean_image

In [27]:
y_train_binary = to_categorical(y_train)
y_test_binary = to_categorical(y_test)

In [28]:
def create_inception_model():
  model = InceptionV3(weights=None, include_top=False, input_shape=(300, 300, 3), pooling='max')
  x = model.output
  predictions = Dense(1, activation='softmax')(x)
  myModel_scratch = Model(inputs=model.input, outputs=predictions, name='InceptionV3')

  opt = keras.optimizers.Adam(learning_rate=0.0001)
  myModel_scratch.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return myModel_scratch

In [40]:
def kfold_2(X_train,y_train,K):

  model_validation=create_inception_model()
  cvscores = []
  i = 1
  skf = KFold(n_splits=K)

  X_train_ = np.array(X_train)
  y_train = np.array(y_train)
  for train_index,val_index in skf.split(X_train_):
    x_train_new,x_val=X_train_[train_index],X_train_[val_index]
    y_train_new,y_val=y_train[train_index],y_train[val_index]

    model_validation.fit(x_train_new, y_train_new, validation_data = (x_val,y_val), epochs=20, verbose=0)

    scores = model_validation.evaluate(x_val, y_val, verbose=0)
    # print("Fold ke",i," -> ",scores[1]*100,"%")
    print("Fold ke",i," -> ",end=" ")
    print("%.3f%%"%(scores[1]*100))
    i=i+1
    cvscores.append(scores[1])

  return cvscores

In [42]:
model = create_inception_model()
model.fit(X_train, y_train_binary,epochs=5)

Epoch 1/5
3/3 [==============================] - 69s 18s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/5
3/3 [==============================] - 61s 18s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
3/3 [==============================] - 61s 18s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
3/3 [==============================] - 61s 18s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
3/3 [==============================] - 61s 18s/step - loss: 0.0000e+00 - accuracy: 1.0000


In [43]:
predictions = model.predict(X_test , batch_size = 32).argmax(axis = 1)
print(classification_report(y_test_binary.argmax(axis = 1), predictions))
print(confusion_matrix(y_test_binary.argmax(axis = 1), predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

[[20]]


In [44]:
accuracy = model.evaluate(X_test, y_test_binary, verbose=1)
print('Test loss    :', accuracy[0])
print('Test accuracy: %.2f%%' % (accuracy[1]*100))

1/1 [==============================] - 5s 5s/step - loss: 0.0000e+00 - accuracy: 1.0000
Test loss    : 0.0
Test accuracy: 100.00%
